In [1]:
%load_ext autoreload
%autoreload 2

import galsim
import numpy as np
from tqdm import trange
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append(
    "/data/groups/jeltema/zhou/lsst_shear/WeakLensingDeblending/bin"
)

In [3]:
import mdet_meas_tools as mmt
from sim_func import sim_func
import galsim
import numpy as np
import pandas as pd
plt.style.use("MNRAS")

NameError: name 'plt' is not defined

In [4]:
gal_flux = 1e12   # total counts on the image
rhalf = 0.5  # arcsec
psf = galsim.Moffat(fwhm=0.7, beta=2.5)
pixel_scale = 0.2  # arcsec / pixel
noise = 520. # standard 

gal = galsim.Exponential(flux=gal_flux, half_light_radius=rhalf)
gal = gal.shear(e1=0.3, e2=-0.8)
# rot_gal= gal.rotate(np.pi/2.* galsim.radians)
gal = gal.withFlux(gal_flux)
final = galsim.Convolve([gal, psf])
image = final.drawImage(scale=pixel_scale, nx=53, ny=53)
image.addNoise(galsim.GaussianNoise(sigma=noise))

In [ ]:
dc2_sub = 

In [8]:
nbins = 4
R11_result = []
for i in trange(nbins):
    gal_list_path = "../data/gal_list_{}.pkl".format(i)
    pdata, mdata, R11, R11sd, m, msd, c, csd = mmt.run_mdet_sims(gal_list_path,
        sim_func=sim_func,
        sim_kwargs={
            "noise": 420
        },
        seed=123,
        n_sims=1000,
        use_p=True,
        use_m=True,
        jackknife=100)
    R11_result.append((R11,R11sd))

  0%|          | 0/4 [00:00<?, ?it/s]

Using both positive and negative shear
x1 is [ 1.07751101e-01  1.87514696e+00  5.95545182e-01 -4.93466604e-01
  6.01389931e-02  3.40094817e-01  1.06509570e+00  1.18166411e-01
  5.26414698e-01  1.45756755e-01  6.24305613e-01  6.98609254e-01
 -3.02283405e-01  3.10088806e-01  4.72519948e-01  5.30155285e-02
  3.87014925e-01  5.38793581e-01  6.96344492e-01 -5.93193131e-01
  6.30502039e-01 -7.00822553e-02  2.91715921e-01  4.02513684e-01
 -4.48168163e-02  5.04165328e-01  2.25041468e-02 -8.91764880e-02
  2.14454528e-01  9.80839658e-02  5.77909570e-01  5.59066184e-01
  5.06773204e-01  1.53380180e-01  9.95039555e-01  6.23494605e-01
 -1.70850900e-01  3.59827452e-01 -6.53056111e-02  4.41354103e-01
 -1.67854688e-01  3.88567212e-01  1.03130724e+00  2.16607823e-01
 -3.83530871e-01  8.05349968e-01  7.56718469e-01  4.67278340e-01
  1.21794341e-01 -2.15433376e-01 -2.41711657e-01 -4.35896430e-01
  2.66459061e-02  3.50123467e-01  5.78417619e-01  3.60909593e-01
  4.39776437e-01 -6.29231775e-01  2.74915742e

 25%|██▌       | 1/4 [02:42<08:07, 162.42s/it]

Using both positive and negative shear
x1 is [-1.24366236e-02  8.72535148e-01  4.67300904e-01 -4.34617933e-01
  8.67355584e-02  2.31947210e-01  8.32769707e-01  3.59417559e-02
  4.18657704e-01  5.52661263e-01  7.00345777e-01 -3.20802100e-01
  1.97451973e-01  2.39081812e-01  3.27626377e-02  4.19387629e-01
  2.07591318e-01 -5.84663990e-01  5.14508242e-01 -1.68662557e-01
  1.46015036e-01  4.64063760e-01 -5.14844636e-02  3.47056116e-01
 -7.73410346e-03 -9.62275222e-02  8.35883677e-02  4.87227838e-02
 -4.62903240e-01  3.95019164e-01  4.48467571e-01 -7.65347391e-02
  5.75693676e-01  3.30147092e-01  1.93043219e-01  3.55590923e-01
 -3.98780111e-02  4.10954563e-01 -1.65665490e-01  2.97042565e-01
  7.08747774e-01  2.10424982e-01  1.03732864e-01  5.82734350e-01
  4.30754450e-01  3.09811596e-01  7.78472999e-02 -2.26016978e-01
 -2.79304321e-01 -3.48272334e-01 -2.79236741e-02  3.31011239e-01
  4.44154636e-01  3.48588113e-01 -4.83833961e-01  1.38387561e-01
  1.66226661e-01 -5.66252330e-01 -2.78680882e

 50%|█████     | 2/4 [05:26<05:27, 163.51s/it]

Using both positive and negative shear
x1 is [-6.99161516e-03  5.88565353e-01  8.16852842e-01 -1.09430136e-01
  2.41213054e-01 -8.47517200e-01 -6.93887137e-01 -1.15900862e-01
  3.95013643e-01 -1.36440439e-01  5.00248589e-01 -2.73486894e-01
  3.92680827e-01 -2.83734405e-01 -4.02334890e-01  7.25049379e-02
  7.08152588e-01 -5.73809807e-02 -7.60016081e-01  3.43682856e-01
 -1.00439154e+00 -5.35207147e-01  4.97250484e-01  3.68786636e-01
 -1.05344799e-01 -3.60398247e-01  2.28142927e-01  1.55474628e-01
  5.39974115e-01  2.38641518e-01  3.84417410e-01  6.66124403e-01
  8.69527908e-01 -4.14418054e-01  1.98419149e-01 -1.86289484e-02
  9.55180881e-01 -5.33514022e-02  2.40867647e-01 -1.82680456e-01
  1.99834595e-01  3.54831016e-01  1.43242566e-03  4.27374301e-01
  3.82060000e-01 -6.50416467e-01  4.03118786e-01 -4.23503437e-01
  3.40824681e-02  9.70345748e-01  2.13332381e-01  4.53062078e-02
 -4.41529692e-01  1.44831046e-04 -4.40079673e-01  3.82237238e-01
  5.06991350e-01  6.71434479e-02 -2.37133443e

 75%|███████▌  | 3/4 [08:08<02:42, 162.91s/it]

Using both positive and negative shear
x1 is [0.21199131 0.36855575 0.26635703 0.09813135 0.22463132 0.23978349
 0.33594243 0.18515187 0.29278314 0.31485525 0.13467712 0.26079101
 0.24607641 0.22489297 0.22235625 0.12993189 0.18348133 0.19726074
 0.28143089 0.21317044 0.19954268 0.2465512  0.21671861 0.26159807
 0.25933019 0.23608721 0.29814308 0.22267244 0.27166901 0.18475115
 0.16782343 0.26402188 0.1659504  0.30088733 0.23812369 0.15621458
 0.1526544  0.20368017 0.24963219 0.27073444 0.26540936 0.1031508
 0.24787938 0.07824086 0.15737233 0.29171869 0.25732814 0.19370688
 0.28379465 0.17403884 0.23588206 0.24302627 0.28707001 0.27550405
 0.24631572 0.23208176 0.14190376 0.25456973 0.2131649  0.24070573
 0.32583438 0.27520645 0.15277422 0.29238569 0.28413243 0.28191829
 0.29121902 0.33926015 0.28030743 0.22550368 0.16329101 0.23068382
 0.2925207  0.22431178 0.33365136 0.34050662 0.23995908 0.21453295
 0.2393581  0.23696859 0.2407583  0.24836793 0.1590844  0.25484938
 0.25305408 0.2106

100%|██████████| 4/4 [10:50<00:00, 162.60s/it]


In [ ]:
import pickle
with open('../output/R11_result.pkl', 'wb') as f:
    pickle.dump(R11_result, f)